# Time Series Analysis with Python/Pandas


## Overview

In this session we're going to move swiftly on from our generic introduction to programming covered last time. By the end of the session, we will have used python to do something genuinely useful -- we will generate a 'day of year climatology' for air temperature at the Loughborough weather station. As discussed, this can be employed as a type of prediction, and we will use it as such to forecast temperatures between now and the next session. Later in the course we will then see if this performance can be improved upon using more sophisticated methods to capture variability arount the climatology (i.e. the 'weather'!) 

To achieve the aim indicated above, we will make use of python $\it{modules}$. These are libraries of code -- written by others -- which we can use to do sophisticated analysis. Use of modules speeds up our work by several orders of magnitude, and enables us to build on the collective skill of tens of thousands of programmers. 

Today, we will mainly be using the "pandas" module, which was created with data scientists in mind. Pandas enables us to use syntax that is far more intuitive than often encountered elsewhere, and it comes loaded with lots of super-helpful functions. We will only scratch the surface of pandas today, but will return to it in later sessions. 

We load the pandas module like this (note: please run code like this as you move through the notebook): 

In [4]:
import pandas as pd 

Note the generic syntax -- "import x as y". This means, basically, 'load' the module x, but let me refer to it as y. We do this because we will access attributes of the module using its name and the "." notation, and it is laborious (and prone to error) if we have to type out the full name of the moudle each time we want to access one of its attributes -- so we use shorthand wherever possible. 

We are also going to import some other modules needed for today's session: 

- Matplotlib (https://matplotlib.org/) -- used for making plots/graphs
- Numpy (https://numpy.org/) -- used for numerical computation in python
- Sklearn (https://scikit-learn.org/stable/) -- used for 'machine learning'

Of these, matplotlib and numpy are modules you will use exceptionally often if continuing to learn python after this course. Sklearn brings the dark art of "machine learning" -- an application of "artifical intelligence" -- to python. We will use if for something far more mundane/familiar today! 

Run the code below to load the modules. Also note that I have defined a function for you in here (called 'seas_cycle') -- it uses methods from the numpy and sklearn to help 'model' the seasonal cycle of a variable (like air temperature). You will use this later, but don't need to understand all the individual bits of code now. 

In [5]:
# These are the 'import' statements
# --------------------------------------------------------------------------
import matplotlib.pyplot as plt # note that here I do something subtly 
# different - I import an attribute of matplotlib -- called 'pyplot'
import pandas as pd
import numpy as np
from sklearn import linear_model # this is a little different again, 
# I'm only importing one attribute (linear_model) from sklearn. 
# --------------------------------------------------------------------------

# Below is the code where I define a function for us to use. You do not need
# to understand this yet.
# --------------------------------------------------------------------------
def seas_cycle(ind,n,indata):
    y=indata.values[:]
    reg = linear_model.LinearRegression() 
    ind = 2*np.pi*(ind/np.float(n)) 
    idx = ~np.isnan(y)
    X = np.column_stack((np.cos(ind[idx]),np.sin(ind[idx])))
    reg.fit(X, y[idx]) 
    sim = reg.predict(np.column_stack((np.cos(ind),np.sin(ind)))) 
    return sim
# --------------------------------------------------------------------------    
    

ModuleNotFoundError: No module named 'sklearn'

The best way to illustrate the power of Pandas is then by example -- as we work towards our goal of computing a "climatology" to make some predictions about the weather. Let's begin by loading data from our very own campus weather stations introduced to you already. You will need to change the path (i.e. the variable 'fin') for this code to work for you -- it will need to reflect where you stored the file.

In [ ]:
fin="C:/Users/gytm3/OneDrive - Loughborough University/Teaching/GYP035/201920/Campus_Met_Data_Student.csv"
date_parser=lambda x: pd.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
data=pd.read_csv(fin,parse_dates=["TIMESTAMP"],date_parser=date_parser,index_col=0)

To be clear, I'm accessing the 'method' (or 'function') 'read_csv' that is part of the pandas module. The 'arguments' or 'parameters' I provide to the function are those comma-separated items within the brackets: the first is the $\it{full}$ filename, the second is a flag that tells pandas that it should try to interpret any dates in the file; and the third is an instruction to tell pandas that the first column of data in the file should be used to index the data -- i.e. which column is the 'axis'. The result of this command is the creation of a new variable called 'data', which is a 'DataFrame' -- a specific type that is available in the pandas module. DataFrames are very clever data types, ideally set up for data analysis. You will see some of these features as we progress through the session. 

How would $\it{you}$ figure out what arguments are needed by this method if you didn't have me here to tell you? Easy, you search the help pages. In this case, googling the 'read_csv' method of the pandas module should take you to this page: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html. Or, alternatively, you can ask python directly:

In [ ]:
help(pd.read_csv)

Personally, I prefer reading in my browser...

OK. So back to why we should get excited about pandas and DataFrames...  Below we're goint to print an attribute of data -- 'columns'. This simply outputs a list -- which corresponds to the headings in our csv file. We do this because it's a good check that the data have been read correctly -- and to make sure we refer to the different variables in a way that pandas will understand.

In [ ]:
print(data.columns)

Running the above should confirm that pandas has understood the structure of your input data. Now, let's immediately do something fun. Let's make a plot of air temperature (which we see is called 'AirTC_Avg'):

In [ ]:
# This first command is needed to make the plots 'appear' in this notebook
%matplotlib inline 
data["AirTC_Avg"].plot()

How easy was that? And look - pandas has recognised that date (labelled as 'TIMESTAMP') should be used as the x-axis! Ha! Take that, Microsoft Excel, and your habit of ruining my life with your trouble processing dates... 

It's important to digest what we have just done: 

- indexing: data["AirTC_Avg"] is another, far more intuitive way of 'indexing' an 'iterable' (covered last session) that pandas enables us to use on the DataFrame type  
- plotting: the .plot() call is us using the plot method of the data["AirTC_Avg"] variable. There are other ways to make plots in python, but none as straightforward for us -- be glad when your data are in a pandas DataFrame!

OK, so that might not seem too impressive; let's do something else to showcase what pandas can do -- and make something more informative to plot at the same time.

Below I'm going to use the 'resample' method to, as you may guess, sub-select (or sample) the data. The resample I'm going to perform is to move from 15-min data (the native format), to daily resolution:

In [ ]:
daily_t=data["AirTC_Avg"].resample("D") # Note that the "D" indiates "Daily".
# Even though the code will be new to you, hopefully you find it relatively 
# intuitive. This readability is a deliberate feature of python

Nothing happened! 

That's right. All that pandas has done is some work behind the scenes to aggregate the data into daily chunks. But $\it{how}$ do we want to summarise the data at this lower (daily) resolution -- where we move from 24 x 4 = 96 values/day to just 1? 

Well, we can choose, by applying another method to the resampler ('daily_t' is the resampler). Below we compute daily mean, min, and max temperatures:

In [ ]:
daymean_t=daily_t.mean()
daymax_t=daily_t.max()
daymin_t=daily_t.min()

Note that above I call a method with the parentheses, but there are no arguments included. This is beacause pandas does not need any more information to calculate the mean, max or min -- just the variable itself (in this case, daily_t). 

daymean_t, daymax_t, and daymin_t are now stand-alone variables containing the daily mean, maximum, and minimum temperature, respectively. To check this, we can plot them:

In [ ]:
daymean_t.plot()
daymax_t.plot()
daymin_t.plot()

Not bad, eh? And look how hot it was this summer! Let's ask pandas to tell us what this 'record' temperature was...

In [ ]:
print("Record high temperature was: ",daymax_t.max(),"deg C")

And how about the min?

In [ ]:
print("Record low temperature was: ",daymin_t.min(),"deg C")

In the above plots, you can, I hope, clearly make out the cyclical nature of the temperatures, with four 'peaks' (summers) and three 'troughs' (winters).  

There are a few ways we can 'model' this behaviour to produde a 'day of year climatology' (hereafter 'DoY climatology'). One of the easiest is to group together all the 'days of the year' and average them. For example, we could find all the January 1sts (there will be 3 of them), and find the average value -- repeating for Jan 2nd, and so forth). The code for this is a bit clunky (or at least, has quite a bit going on that you may not understand), but execute the below to generate a DoY climatology -- using mean temperatures for our example. 

In [ ]:
doy=daymean_t.index.dayofyear # finds the DoY from the date/time axis
doy_clim=daymean_t.groupby(doy).mean() # ** finds mean value for the all those unique DoYs
# line below puts the correct DoY mean value at the right date
doy_mean=[doy_clim[doy_clim.index==i].values[0] for i in doy.values[:]] # 

Before we look at the output of that, let's estimate the DoY climatology another way -- using a sine wave to approximate the smooth undulation. For those of you unfamiliar with a sine wave, here's the world's briefiest overview of what it is:

... A sine wave is the curve that is produced by taking the "sine" of an ordered sequence of angles between 0 and 360 degrees (or 2$\pi$ radians). The "sine" of argument $x$, remember, is just the ratio of the 'opposite' (vertical) to 'hypotenuse' sides in a right angle triangle, when the angle between the hypotenuse and adjacent sides of the triangle is equal to $x$...

This isn't a trigonmetry lesson, so don't worry if you've forgotten what the sine function does, and you don't understand my explanation above. We can instead just think of a sine wave as a function that creates a periodic (repeating), smooth 'wave' form. As discussed in the lecture, there is good physical reason why we expect that this particular shape should describe our temperature data well.

In the below, I demonstrate a sine wave, creating a sequence of numbers to represent the angles 0-360 (using the 'arange' method of numpy); I then feed this to the sine method of numpy (called 'sin') and plot the result for you to see what it looks like. Don't be confused by the 'radians' command -- this just converts the angles into the form that numpy expects. 

In [ ]:
angles=np.radians(np.arange(0,361)) # creates the reference series
y=np.sin(angles) # use the sin function
plt.plot(y) # this is another way to plot something when it isn't a DataFrame

Nice shape, right? This has the smooth undulations just as we wanted them. However, how do we get the 'height' (amplitude) to be correct, and how do we make sure that the peak is positioned in summer and the trough in winter (i.e. that the wave has the correct 'phase'? These are important questions, but beyond the scope of our session, so I have written a function for you (the 'seas_cycle' function) to do this. Essentially, it tries to make the curve fit *all* of the data as closely as possible -- i.e passing as closely as possible to all observed temperatures. 

Run the code below to generate a DoY climatology using this function. 

In [ ]:
clim=seas_cycle(doy.astype(np.float),365.25,daymean_t) # **

OK. So we are now in a position to visually compare doy_mean (the simple averaging) and clim (the sine wave) to the observations. In the code below we will create a new pandas DataFrame, inserting the observations and both models, before making a plot using the convenient .plot() method of a DataFrame. Note that here I do plot a little differently this time -- creating a blank figure and pair of axes and then plotting on top.

In [ ]:
out=pd.DataFrame(data={"obs":daymean_t.values[:],"doy_mean":doy_mean,"clim":clim},\
                 index=daymean_t.index) # ** creating a DataFrame
fig,ax=plt.subplots(1,1) # blank figure/axes to plot on
out.plot(ax=ax) # normal call to plot -- but specifying 'ax' to plot on
ax.set_ylabel("Air temperature (deg C)") # cosmetics -- label y-axis
ax.set_xlabel("Date") # cosmetics -- label x-axis

We can now save the DataFrame 'out' to a .csv file (which can be opened in Excel -- and read back in using python). We will do this so that you can explore the data outside of the notebook -- and so that you can easily build on this dataset in the future. Change 'fout' in the code below (to indicate the path of the file you want to save the data in), and then execute the code block. 

In [ ]:
fout="C:/Users/gytm3/OneDrive - Loughborough University/Teaching/GYP035/201920/DoY_climatology_tmean.csv" # **
out.to_csv(fout)

### Your challenge -- to be completed before the next practical

- Edit the code segments above so that you repeat the modelling for the daily min and daily maximum temperatures. [Hint: this is not as tricky as it seems -- look for bits of code with # ** -- these are the only bits you need to edit]


- Use the output (.csv files) to estimate:


    - the date most likely to record the annual maximum temperature 
    - the date most likely to record the annual minimum temperature
    - the mean temperature on the day of our next practical (Monday)
    

- Finally, which method (doy_mean or clim) do you think does a 'better' job of capturing the observed air temperature variability? Suggest ways in which we could 'score' the different methods. 


In [6]:
import statsmodels.api as sm